In [25]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pathlib import Path

# Load environment variables from .env file
current_dir = os.getcwd()
env_path = os.path.join(current_dir, '.env')
# Access your API key from the .env file
load_dotenv(dotenv_path=env_path)  
api_key = os.getenv("API_KEY")
# Initialize the model with the API key
model = ChatOpenAI(model="llama-3.3-70b-versatile", api_key=api_key, base_url="https://api.groq.com/openai/v1")
print(api_key)
print(env_path)

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
c:\Users\Admin-62501\Desktop\Coding\Projects\Customer agents for market simulation\agent\.env


我們也可以自己寫agent: https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/


`config` object in LangGraph Swarm, specifically the {"configurable": {"thread_id": "1"}} structure, is not modified by the system during execution. It serves as a persistent identifier that you provide to maintain conversation state across multiple interactions

`router` in `StateGraph`: router refers to a special type of node or function that determines the flow of execution through the graph based on the current state. It acts as a decision-making component that directs which node should be executed next.

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

alice = create_react_agent(
    model,
    [add, create_handoff_tool(agent_name="Bob")],
    prompt="You are Alice, an addition expert.",
    name="Alice",
)

bob = create_react_agent(
    model,
    [create_handoff_tool(agent_name="Alice", description="Transfer to Alice, she can help with math")],
    prompt="You are Bob, you speak like a pirate.",
    name="Bob",
)
# short-term memory
# maintain conversation state across interactions
checkpointer = InMemorySaver()
# long-term memory
store = InMemoryStore()

workflow = create_swarm(
    [alice, bob],
    default_active_agent="Alice"
)
# Compiles the state graph into a CompiledStateGraph object.
app = workflow.compile(
    checkpointer=checkpointer,
    store=store
)
# 
config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "i'd like to speak to Bob"}]},
    config,
)
print(turn_1)
turn_2 = app.invoke(
    {"messages": [{"role": "user", "content": "what's 5 + 7?"}]},
    config,
)
print(turn_2)

{'messages': [HumanMessage(content="i'd like to speak to Bob", additional_kwargs={}, response_metadata={}, id='b35e6d5a-3a62-46c2-8ca8-0a4b945b3b94'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2966', 'function': {'arguments': '{}', 'name': 'transfer_to_bob'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 275, 'total_tokens': 287, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.262570968, 'prompt_time': 0.024186555, 'completion_time': 0.043636364, 'total_time': 0.067822919}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'id': 'chatcmpl-2324f17b-a548-40ad-a1e7-c760ebfb415d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, name='Alice', id='run--6bd1fe6e-6074-4935-80da-7399e5f7cf0a-0', tool_calls=[{'name': 'transfer_to_bob', 'args': {}, 'id': 'call_2966', 'type': 'tool_call'}], usage_metadata={'input_t

Interactive agents

In [30]:
def format_agent_response(response):
    """Format the agent response for better readability."""
    agent_name = response.get("active_agent", "Unknown")
    messages = response.get("messages", [])

    if not messages:
        return f"[{agent_name}] No response."

    last_message = messages[-1]
    content = getattr(last_message, 'content', "")  # Access 'content' attribute directly

    return f"[{agent_name}] {content}"

def interactive_chat():
    """Run an interactive chat session with the agent swarm"""
    print("Welcome to the Agent Swarm Chat!")
    print("Type 'exit' or 'quit' to end the conversation.")
    print("Starting with Alice as the default agent.")
    print("-" * 50)
    
    while True:
        # Get user input
        user_input = input("You: ")
        
        # Check if user wants to exit
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        
        # Create a message with just the current user input
        # The checkpointer will maintain the conversation state
        current_message = [{"role": "user", "content": user_input}]
        
        # Invoke the agent swarm with just the new message
        response = app.invoke(
            {"messages": current_message},
            config,
        )
        
        # Extract and display the agent's response
        formatted_response = format_agent_response(response)
        print(formatted_response)


In [31]:
interactive_chat()

Welcome to the Agent Swarm Chat!
Type 'exit' or 'quit' to end the conversation.
Starting with Alice as the default agent.
--------------------------------------------------
[Bob] Yer already talkin' to me, matey! I be Bob, the pirate. What be on yer mind? Want to discuss some treasure or maybe sing a sea shanty?
[Alice] Hello, I'm Alice. I can help with any addition problems you may have. What's on your mind?
[Alice] My name is Alice. I'm an addition expert, which means I can help with any questions you have about adding numbers.
[Alice] The answer to 1 + 1 is 2.
[Alice] Hello! It's nice to meet you. I'm Alice, the addition expert. Is there a math problem you'd like help with?
[Bob] Ahoy, matey! Welcome back to me ship. I be Bob, the scurvy dog. What be bringin' ye to these fair waters? Do ye have a treasure to share or a tale to tell?
[Bob] I be a pirate, not a weather forecastin' landlubber. Me knowledge be limited to the seven seas and me trusty map. If ye want to know the weather, 